In [1]:
import sys

sys.path.append("../lfit_gui/")

import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.colors as colors
from astropy.io import fits
from astropy.visualization import simple_norm
from astropy.visualization import PercentileInterval, AsinhStretch
from astropy.wcs import WCS
import os, commands
import lens_funcs_gui as lf
import sys

In [2]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.utils.data import clear_download_cache

plt.rcParams.update({'font.sans-serif': 'Computer Modern Sans Serif', 'mathtext.fontset': 'cm', 'font.size': 8, 'font.weight': 'normal', 'font.family': 'serif', 'mathtext.fontset': 'dejavuserif', 'xtick.labelsize': 8, 'ytick.labelsize': 8, 'xtick.major.width': 1.0, 'xtick.major.size': 6.0, 'xtick.minor.size': 3.0, 'ytick.major.width': 1.0, 'ytick.major.size': 6.0, 'ytick.minor.size': 3.0, 'axes.linewidth': 1.0, 'legend.borderpad': 0.2, 'legend.handletextpad': 0.2, 'legend.labelspacing': 0.4, 'interactive': False})

'''
Assume you have these four data
1. cutout: the cutout of the system
2. cutout_invvar: inverse variance of cutout
3. star: the cutout of the star
4. star_invvar: inverse variance of star
'''



'\nAssume you have these four data\n1. cutout: the cutout of the system\n2. cutout_invvar: inverse variance of cutout\n3. star: the cutout of the star\n4. star_invvar: inverse variance of star\n'

In [45]:
! ls

data	     fit_based_on_firstband.ipynb  model_plot.py       未命名.ipynb
DLM.ipynb    fit_firstband.ipynb	   results
DLM_results  model_band_plot.py		   show_results.ipynb


In [55]:
band="r"
band_basedon="z"
obj="J0746"
obj_name=obj+"-"+band

#g-12
#r-34
#z-56

if band=="g":
    hdul = fits.open('./data/'+obj+'/'+obj+'-cutout.fits')
    cutout = hdul[1].data 
    cutout_invvar = hdul[2].data

    hdul = fits.open('./data/'+obj+'/'+obj+'-star.fits')
    star = hdul[1].data
    star_invvar = hdul[2].data
elif band=="r":
    hdul = fits.open('./data/'+obj+'/'+obj+'-cutout.fits')
    cutout = hdul[3].data 
    cutout_invvar = hdul[4].data

    hdul = fits.open('./data/'+obj+'/'+obj+'-star.fits')
    star = hdul[3].data
    star_invvar = hdul[4].data
elif band=="z":
    hdul = fits.open('./data/'+obj+'/'+obj+'-cutout.fits')
    cutout = hdul[5].data 
    cutout_invvar = hdul[6].data

    hdul = fits.open('./data/'+obj+'/'+obj+'-star.fits')
    star = hdul[5].data
    star_invvar = hdul[6].data
    
else:
    assert False
    
#0746
n_PSF = 2
x_PSFs = [-5,1]
y_PSFs = [4, -5]
peak_PSF = [2.4567146,0.6257189]

#2121
n_PSF = 2
x_PSFs = [1,-1]
y_PSFs = [1, -1]
peak_PSF = [10,10]

In [56]:
# Some basic parameters
# print(cutout.shape)
hw=cutout.shape[0]/2
hw_star=star.shape[0]/2
dpix=1
pixel_to_arcsec=0.262


mask=np.ones_like(star, dtype=int)
hdu=fits.PrimaryHDU(1.0*star)
hdulist=fits.HDUList([hdu])
hdulist.append(fits.ImageHDU(star_invvar))
hdulist.append(fits.ImageHDU(np.array([0])))
hdulist.append(fits.ImageHDU(mask))
hdulist.writeto('temp_star.fits', overwrite=True)

In [57]:
#read previous result
prior_result_path = './results/'+obj+'-'+band_basedon+'_fit_PSFs_one_sersic.fits'
hdul = fits.open(prior_result_path)
ppar0=(hdul[3].data).p_phot

#PSF0
psf0_xcen=ppar0[1] #xcen
psf0_ycen=ppar0[2] #ycen

#PSF1
psf1_xcen=ppar0[8] #xcen
psf1_ycen=ppar0[9] #ycen

#PSF2
psf2_xcen=ppar0[15] #xcen
psf2_ycen=ppar0[16] #ycen

#PSF3
psf3_xcen=ppar0[22] #xcen
psf3_ycen=ppar0[23] #ycen

#sersic
sersic_xcen=ppar0[29] #xcen
sersic_ycen=ppar0[30] #ycen
sersic_r=ppar0[31]  #q
sersic_pa=ppar0[32] #pa
sersic_q=ppar0[33]  #q
sersic_n=ppar0[34]  #q

In [58]:
psf0_xcen,psf0_ycen

(-3.997046897986724, 4.194647613457185)

In [59]:
psf2_xcen,psf2_ycen

(0.3599058835429112, -4.05924698540589)

## 拟合恒星

In [9]:
f=open('parfile_star', 'w')
f.write("temp_star.fits 0\n")
f.write("temp_star.fits 1\n")
f.write("temp_star.fits 2\n")
f.write("temp_star.fits 3\n")
f.write("temp_star.fits 3\n")
f.write("temp_star.fits 3\n")
f.write("star_fit.fits\n")
f.write("{:d} {:f}\n".format(hw_star, dpix))
f.write("0  sersic\n")
f.write("0  sie\n")
f.write("3  moffat,moffat,foreg\n")
f.write("1\n")
f.write("0\n")#scale

f.write("value={:f}, min=0.0\n".format(star.sum()-np.median(star)*hw_star**2.0*4))
f.write("value=0.0, min=-3.0, max=3.0\n")
f.write("value=0.0, min=-3.0, max=3.0\n")
f.write("value=3.0, min=0.01\n")
f.write("value=120.0, vary=1, min=0.0, max=180.0\n")
f.write("value=0.8, vary=1, min=0.1, max=1.0\n")
f.write("value=2.0, min=1.0, max=10.0\n")

f.write("value={:f}, min=0.0\n".format(star.sum()-np.median(star)*hw_star**2.0*4))
# f.write("expr=phot0_xcen\n")
# f.write("expr=phot0_ycen\n")
f.write("value=0.0, min=-3.0, max=3.0\n")
f.write("value=0.0, min=-3.0, max=3.0\n")
f.write("value=3.0, min=0.01\n")
f.write("value=120.0, vary=1, min=0.0, max=180.0\n")
f.write("value=0.8, vary=1, min=0.1, max=1.0\n")
f.write("value=2.0, min=1.0, max=10.0\n")

f.write("value={:f}\n".format(np.median(star)))
f.close()

### 激活py2，转移notebook所在目录后，复制以下命令到终端执行：
#### ../lfit_gui/lfit_script  ./parfile_star

## 拟合twoPSF + sersic

In [60]:
flag=1   #默认使用刚刚拟合的恒星
try:
    hdulist=fits.open("star_fit.fits")
except:
    flag=0  #如果读取不到，说明使用之前拟合过的恒星。也就是说，不重新拟合。
    hdulist=fits.open("./results/"+obj+"-"+band+"_fit_star.fits")
ppar=(hdulist[3].data).p_phot

peak_moffat_1=ppar[0]
x_moffat_1=ppar[1]
y_moffat_1=ppar[2]
alpha_moffat_1=ppar[3]
pa_moffat_1=ppar[4]
q_moffat_1=ppar[5]
beta_moffat_1=ppar[6]

peak_moffat_2=ppar[7]
x_moffat_2=ppar[8]
y_moffat_2=ppar[9]
alpha_moffat_2=ppar[10]
pa_moffat_2=ppar[11]
q_moffat_2=ppar[12]
beta_moffat_2=ppar[13]

dx_moffat = x_moffat_2 - x_moffat_1
dy_moffat = y_moffat_2 - y_moffat_1
fpeak_moffat = peak_moffat_2/peak_moffat_1

hdulist.close()

#make 2 psf become one

x=np.outer(np.ones(2*hw+1), np.arange(2*hw+1)-hw)*dpix
y=np.outer(np.arange(2*hw+1)-hw, np.ones(2*hw+1))*dpix
PSF1=lf.moffat_2d(x, y, np.array([1.0, 0.0, 0.0, alpha_moffat_1, pa_moffat_1, q_moffat_1, beta_moffat_1]))
PSF2=lf.moffat_2d(x, y, np.array([fpeak_moffat, dx_moffat, dy_moffat, alpha_moffat_2, pa_moffat_2, q_moffat_2, beta_moffat_2]))
PSF=PSF1+PSF2
PSF=PSF/PSF.sum()

#prepare fitting data
mask=np.ones_like(cutout, dtype=int)
hdu=fits.PrimaryHDU(1.0*cutout)
hdulist=fits.HDUList([hdu])
hdulist.append(fits.ImageHDU(cutout_invvar))
hdulist.append(fits.ImageHDU(PSF))
hdulist.append(fits.ImageHDU(mask))
hdulist.writeto('temp_image.fits', overwrite=True)
np.save(obj_name+"PSF.npy",PSF)

In [61]:
#---------------------------------------two psf + one sersic begin-------------------------------------------------------#

#prepare fitting data
mask=np.ones_like(cutout, dtype=int)
PSF=mask
hdu=fits.PrimaryHDU(1.0*cutout)
hdulist=fits.HDUList([hdu])
hdulist.append(fits.ImageHDU(cutout_invvar))
hdulist.append(fits.ImageHDU(PSF))
hdulist.append(fits.ImageHDU(mask))
hdulist.writeto('temp_image.fits', overwrite=True)



f=open('parfile_PSFs_one_sersic', 'w')
f.write("temp_image.fits 0\n")
f.write("temp_image.fits 1\n")
f.write("temp_image.fits 2\n")
f.write("temp_image.fits 3\n")
f.write("temp_image.fits 3\n")
f.write("temp_image.fits 3\n")
f.write(obj_name+"_fit_PSFs_one_sersic.fits\n")
f.write("{:d} {:f}\n".format(hw, dpix))
f.write("0  sersic\n")
f.write("0  sie\n")
f.write("{:d} ".format(2*n_PSF+2))
f.write(("moffat,"*2*n_PSF)[0:-1]+",sersic,foreg\n")
f.write("1\n")
f.write("1\n")

#write parfile_PSFs - 1st PSF
f.write("value={:f}, vary=1, min=0.0\n".format(peak_PSF[0]))
f.write("value={:f}, vary=1, min={:f}, max={:f}\n".format(psf0_xcen, psf0_xcen-5.0, psf0_xcen+5.0))
f.write("value={:f}, vary=1, min={:f}, max={:f}\n".format(psf0_ycen, psf0_ycen-5.0, psf0_ycen+5.0))
f.write("value={:f}, vary=0, min=0.01\n".format(alpha_moffat_1))
f.write("value={:f}, vary=0, min=0.0, max=180.0\n".format(pa_moffat_1))
f.write("value={:f}, vary=0, min=0.1, max=1.0\n".format(q_moffat_1))
f.write("value={:f}, vary=0, min=1.0, max=10.0\n".format(beta_moffat_1))

#write parfile_PSFs - 2nd PSF
f.write("expr=phot0_amp*{:f}\n".format(fpeak_moffat))
f.write("expr=phot0_xcen+{:f}\n".format(dx_moffat))
f.write("expr=phot0_ycen+{:f}\n".format(dy_moffat))
f.write("value={:f}, vary=0, min=0.01\n".format(alpha_moffat_2))
f.write("value={:f}, vary=0, min=0.0, max=180.0\n".format(pa_moffat_2))
f.write("value={:f}, vary=0, min=0.1, max=1.0\n".format(q_moffat_2))
f.write("value={:f}, vary=0, min=1.0, max=10.0\n".format(beta_moffat_2))

#write parfile_PSFs - 3rd PSF
f.write("value={:f}, vary=1, min=0.0\n".format(peak_PSF[1]))
f.write("value={:f}, vary=1, min={:f}, max={:f}\n".format(psf2_xcen, psf2_xcen-7.0, psf2_xcen+7.0))
f.write("value={:f}, vary=1, min={:f}, max={:f}\n".format(psf2_ycen, psf2_ycen-7.0, psf2_ycen+7.0))
f.write("value={:f}, vary=0, min=0.01\n".format(alpha_moffat_1))
f.write("value={:f}, vary=0, min=0.0, max=180.0\n".format(pa_moffat_1))
f.write("value={:f}, vary=0, min=0.1, max=1.0\n".format(q_moffat_1))
f.write("value={:f}, vary=0, min=1.0, max=10.0\n".format(beta_moffat_1))

#write parfile_PSFs - 4th PSF
f.write("expr=phot2_amp*{:f}\n".format(fpeak_moffat))
f.write("expr=phot2_xcen+{:f}\n".format(dx_moffat))
f.write("expr=phot2_ycen+{:f}\n".format(dy_moffat))
f.write("value={:f}, vary=0, min=0.01\n".format(alpha_moffat_2))
f.write("value={:f}, vary=0, min=0.0, max=180.0\n".format(pa_moffat_2))
f.write("value={:f}, vary=0, min=0.1, max=1.0\n".format(q_moffat_2))
f.write("value={:f}, vary=0, min=1.0, max=10.0\n".format(beta_moffat_2))

#write sersic parameter

f.write("value=5.0, vary=1, min=0.0\n")  #peak value of sersic
f.write("value={:f}, vary=0, min={:f}, max={:f}\n".format(sersic_xcen,sersic_xcen-0.1,sersic_xcen+0.1)) #location x of sersic
f.write("value={:f}, vary=0, min={:f}, max={:f}\n".format(sersic_ycen,sersic_ycen-0.1,sersic_ycen+0.1)) #location y of sersic
f.write("value={:f}, vary=1, min={:f}, max={:f}\n".format(sersic_r,sersic_r-1,sersic_r+0.5))  #effective radius
f.write("value={:f}, vary=0, min={:f},, max={:f},\n".format(sersic_pa,sersic_pa-20,sersic_pa+20)) #position angle
f.write("value={:f}, vary=1, min={:f}, max={:f}\n".format(sersic_q, sersic_q-0.1, sersic_q+0.1))  #q
f.write("value={:f}, vary=1, min={:f}, max={:f}\n".format(sersic_n, sersic_n-0.1, sersic_n+0.1)) #sersic index
f.write("value=0.01, vary=1, min=0.0\n")#background
f.close()



#---------------------------------------two psf + one sersic end-------------------------------------------------------#

### 激活py2，转移notebook所在目录后，复制以下命令到终端执行：
#### ../lfit_gui/lfit_script  ./parfile_PSFs_one_sersic

## 清理文件

In [62]:
if flag==1:
    cmd = "mv ./star_fit.fits  ./results/"+obj_name+"_fit_star.fits"
    print(cmd)
    junk=os.system(cmd)
    assert junk==0

In [63]:
cmd = "mv ./"+obj_name+"_fit_PSFs_one_sersic.fits"+" ./results/"
print(cmd)
junk=os.system(cmd)
assert junk==0

mv ./J0746-r_fit_PSFs_one_sersic.fits ./results/


In [64]:
cmd = "mv "+obj_name+"PSF.npy"+"  ./results/"
print(cmd)
junk=os.system(cmd)
assert junk==0

mv J0746-rPSF.npy  ./results/


In [65]:
cmd="rm temp*"
print(cmd)
junk=os.system(cmd)
assert junk==0

rm temp*


In [66]:
cmd="rm parfile*"
print(cmd)
junk=os.system(cmd)
assert junk==0

rm parfile*
